In [7]:
#!pip install torchaudio librosa matplotlib numpy pandas
#!pip install wget


In [ ]:
#!git clone https://github.com/jaywalnut310/vits.git


In [7]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.lo

In [ ]:
import os
import shutil
from datasets import load_dataset

# ✅ Load dataset from local cache (NO re-downloading)
dataset = load_dataset("erenfazlioglu/turkishvoicedataset", cache_dir="/home/ahmet/.cache/huggingface/datasets")

# ✅ Define the base directory where audio files are stored
dataset_cache_dir = "/home/ahmet/.cache/huggingface/datasets/erenfazlioglu___turkishvoicedataset/default/0.0.0/13238c462f32f6c2fd8293f732f2eac2a03ce48c"

# ✅ Create a directory for extracted audio files
output_dir = "turkish_audio"
os.makedirs(output_dir, exist_ok=True)

# ✅ Copy audio files to a new folder
for i, entry in enumerate(dataset["train"]):
    relative_audio_path = entry["audio"]["path"]  # e.g., '0000001.mp3'
    full_audio_path = os.path.join(dataset_cache_dir, relative_audio_path)  # Convert to full path

    if not os.path.exists(full_audio_path):
        print(f"❌ Missing file: {full_audio_path}")  # Check if file is missing
        continue

    # ✅ Save with a new name
    new_audio_file = os.path.join(output_dir, f"audio_{i}.mp3")
    shutil.copy(full_audio_path, new_audio_file)

    print(f"✅ Copied: {new_audio_file}")

print(f"\n🎉 Extraction complete! Check the 'turkish_audio/' folder.")


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [38]:
#torch.cuda.empty_cache()

In [9]:
tts.tts_to_file(text="I want to meet with you yesterday evening.",
                file_path="ahmet_output_1.wav",
                speaker_wav="ahmt.wav",
                language="en")



 > Text splitted to sentences.
['I want to meet with you yesterday evening.']
 > Processing time: 0.7659907341003418
 > Real-time factor: 0.23727376498809474


'ahmet_output_1.wav'

Hello teammate. Everything is ok. Only I could not find additional English dataset yet. as you know we have 55h English, ~230h DE and with last added ~80h turkish dataset:
so 82k DE, 69k en 45k Tr audio files
What can we do

In [2]:
import json
import os

def parse_metadata_with_wavs(metadata_path, wavs_folder):
    """
    Reads a metadata.txt file and aligns transcriptions with actual wav files.
    
    Now includes **speaker names** in the final dictionary.

    Returns:
    - A dictionary {wav_filename: {"text": transcription, "speaker": speaker_name}}
    """
    speaker_name = os.path.basename(os.path.dirname(metadata_path))  # ✅ Extract speaker from folder name

    # ✅ Step 1: Extract Transcriptions (Order Only)
    transcriptions = []
    with open(metadata_path, "r", encoding="utf-8") as f:
        current_text = []
        for line in f:
            line = line.strip()

            if "|" in line:
                # ✅ New entry → Save the last one
                if current_text:
                    transcriptions.append(" ".join(current_text).strip())

                # ✅ Extract transcription only (ignore filename)
                parts = line.split("|", 1)
                current_text = [parts[1].strip()] if len(parts) > 1 else []
            else:
                # ✅ Continuation of the previous line
                current_text.append(line.strip())

        # ✅ Save the last transcription
        if current_text:
            transcriptions.append(" ".join(current_text).strip())

    # ✅ Step 2: Extract `.wav` filenames in order
    wav_files = sorted([f for f in os.listdir(wavs_folder) if f.endswith(".wav")])

    # ✅ Step 3: Zip & Align
    if len(wav_files) != len(transcriptions):
        print(f"⚠ Warning: Mismatch in number of WAV files ({len(wav_files)}) vs transcriptions ({len(transcriptions)}) in {metadata_path}")

    aligned_data = {wav: {"text": transcript, "speaker": speaker_name} for wav, transcript in zip(wav_files, transcriptions)}

    return aligned_data




In [ ]:
import json
import os

# ✅ Root directory for processed dataset
ROOT_DIR = "m_ailabs_temp/de"
OUTPUT_FILE = "m_ailabs_transcriptions_with_speakers.json"

# ✅ Store all speakers' data
all_speakers_data = {}

# ✅ Loop over each speaker's folder
for speaker_folder in os.listdir(ROOT_DIR):
    speaker_path = os.path.join(ROOT_DIR, speaker_folder)

    # ✅ Ensure it's a folder
    if not os.path.isdir(speaker_path):
        continue

    # ✅ Find all metadata.txt and corresponding wavs folders
    metadata_files = {}
    wav_folders = {}

    for item in os.listdir(speaker_path):
        item_path = os.path.join(speaker_path, item)

        if item.endswith("_metadata.txt"):  # ✅ Identify metadata files
            situation_name = item.replace("_metadata.txt", "")  # Extract situation name
            metadata_files[situation_name] = item_path

        elif item.endswith("_wavs"):  # ✅ Identify wav folders
            situation_name = item.replace("_wavs", "")  # Extract situation name
            wav_folders[situation_name] = item_path

    # ✅ Now we process each situation (aligning metadata with wavs)
    for situation, metadata_file in metadata_files.items():
        if situation in wav_folders:  # ✅ Ensure matching wavs exist
            wavs_folder = wav_folders[situation]
            speaker_data = parse_metadata_with_wavs(metadata_file, wavs_folder)
            all_speakers_data.update(speaker_data)

print(f"✅ Processed {len(all_speakers_data)} audio files across all speakers.")

# ✅ Save full dataset to JSON
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_speakers_data, f, ensure_ascii=False, indent=4)

print(f"✅ All speakers' transcriptions saved to: {OUTPUT_FILE}")


### 1-) SET EACH LANGUAGE TO PROPER JSON ALONG WITH AUDIO INFO, TRANSCRIPTION INFO AND SPEAKERS INFO

In [ ]:
import json
import os
import csv
import sys

# ✅ Root directory containing all languages
ROOT_DIR = "m_ailabs_temp"
OUTPUT_FILE = "lookup.json"

# ✅ Store all data (keyed by language)
all_languages_data = {}
csv.field_size_limit(sys.maxsize)
def parse_metadata_with_wavs(metadata_path, wavs_folder, file_type="txt"):
    """
    Reads a metadata file (TXT or CSV) and aligns transcriptions with WAV files.
    
    - "emre" uses filename-based mapping (because transcript order doesn't match WAV order).
    - All other speakers use the original order-based approach.
    """
    speaker_name = os.path.basename(os.path.dirname(metadata_path))  # ✅ Extract speaker from folder name
    transcriptions = []
    filename_mapping = {}

    if file_type == "txt":
        with open(metadata_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if "|" in line:
                    parts = line.split("|", 1)
                    if len(parts) == 2:
                        filename = parts[0].strip()
                        text = parts[1].strip()
                        transcriptions.append(text)
                        filename_mapping[filename] = text

    elif file_type == "csv":
        with open(metadata_path, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="|")
            for row in reader:
                if len(row) < 2:
                    continue
                filename = row[0].strip()
                text = row[1].strip()
                transcriptions.append(text)
                filename_mapping[filename] = text

    # ✅ Get WAV filenames
    wav_files = sorted([f for f in os.listdir(wavs_folder) if f.endswith(".wav")])

    # ✅ Special case for "emre": Match filenames instead of order
    if speaker_name == "emre":
        aligned_data = {}
        for wav in wav_files:
            if wav in filename_mapping:
                aligned_data[wav] = {
                    "text": filename_mapping[wav],
                    "speaker": speaker_name
                }
            else:
                print(f"⚠ Warning: No transcript found for {wav} in {metadata_path}")
        return aligned_data

    # ✅ For all other speakers: Use the order-based approach
    if len(wav_files) != len(transcriptions):
        print(f"⚠ Warning: Mismatch in {metadata_path} → WAVs: {len(wav_files)}, Transcripts: {len(transcriptions)}")

    return {
        wav: {
            "text": transcript,
            "speaker": speaker_name
        }
        for wav, transcript in zip(wav_files, transcriptions)
    }


# ✅ Process Each Language
for language_folder in os.listdir(ROOT_DIR):
    lang_path = os.path.join(ROOT_DIR, language_folder)

    if not os.path.isdir(lang_path):
        continue  # ✅ Skip non-folder items

    print(f"🔹 Processing Language: {language_folder}")

    all_languages_data[language_folder] = {}

    # ✅ Process Each Speaker in the Language
    for speaker_folder in os.listdir(lang_path):
        speaker_path = os.path.join(lang_path, speaker_folder)

        if not os.path.isdir(speaker_path):
            continue

        print(f"   🎤 Speaker: {speaker_folder}")

        metadata_files = {}
        wav_folders = {}

        # ✅ Find Metadata & Wavs
        for item in os.listdir(speaker_path):
            item_path = os.path.join(speaker_path, item)

            if item.endswith("_metadata.txt"):
                situation_name = item.replace("_metadata.txt", "")
                metadata_files[situation_name] = (item_path, "txt")

            elif item.endswith("_metadata.csv"):
                situation_name = item.replace("_metadata.csv", "")
                metadata_files[situation_name] = (item_path, "csv")

            elif item.endswith("_wavs"):
                situation_name = item.replace("_wavs", "")
                wav_folders[situation_name] = item_path

        # ✅ Process Each Speaker’s Situations
        for situation, (metadata_file, file_type) in metadata_files.items():
            if situation in wav_folders:
                wavs_folder = wav_folders[situation]

                # ✅ Get Transcription Data
                speaker_data = parse_metadata_with_wavs(metadata_file, wavs_folder, file_type)

                # ✅ Store Under Language → Speaker
                all_languages_data[language_folder].update(speaker_data)
                

print(f"✅ Processed {sum(len(data) for data in all_languages_data.values())} audio files across all languages.")

# ✅ Save to JSON
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_languages_data, f, ensure_ascii=False, indent=4)

print(f"✅ Dataset saved: {OUTPUT_FILE}")




🔹 Processing Language: de
   🎤 Speaker: rebeca
   🎤 Speaker: karlson
   🎤 Speaker: ramona
   🎤 Speaker: eva
🔹 Processing Language: tr
   🎤 Speaker: hakan
   🎤 Speaker: emre
   🎤 Speaker: nezus
   🎤 Speaker: hulya
   🎤 Speaker: ruya
   🎤 Speaker: tolga
   🎤 Speaker: melek
   🎤 Speaker: baran
   🎤 Speaker: ahmet
🔹 Processing Language: en
   🎤 Speaker: test_noise
   🎤 Speaker: elliot
   🎤 Speaker: mary
⚠ Warning: Mismatch in m_ailabs_temp/en/mary/mary_northandsouth_metadata.csv → WAVs: 9242, Transcripts: 9243
   🎤 Speaker: elizabeth
   🎤 Speaker: judy
✅ Processed 195657 audio files across all languages.
✅ Dataset saved: m_ailabs_dataset.json


### 2-) ALIGNMENT FOR AUDIO AND TRANSCRIPT TEXT

##### A- Load whisperX

In [ ]:
import whisperx
import torch
from tqdm import tqdm
import os
import argparse


device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16"


# Load the model and aligner
def load_model_with_location(model_name, device, compute_type):
    """Loads the model and prints its location."""
    print(f"Loading model '{model_name}' with compute type '{compute_type}'...")
    cache_dir = os.path.expanduser("~/.cache/whisperx-models")
    model = whisperx.load_model(model_name, device, compute_type=compute_type, download_root=cache_dir)
    print(f"Model loaded from: {cache_dir}")
    return model

# Load the model, and print the model location
model = load_model_with_location("medium", device, compute_type=compute_type)


""" audio = whisperx.load_audio(audio_path)

# Transcribe and align the audio.
result = model.transcribe(audio)

model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result) """

#### Control transcript Quality with Cosine Check

In [2]:
DATASET_PATH = "m_ailabs_temp" 
def find_wav_file(file_name, dataset, language):
    """Finds the correct speaker folder and corresponding wav file path."""
    if language not in dataset:
        print(f"⚠ Warning: Language '{language}' not found in dataset JSON!")
        return None

    if file_name not in dataset[language]:
        print(f"⚠ Warning: File {file_name} not found in dataset[{language}]!")
        return None  # Skip this file

    speaker_name = dataset[language][file_name]["speaker"]
    speaker_path = os.path.join(DATASET_PATH, language, speaker_name)

    # ✅ Check all folders inside the speaker directory
    for folder in os.listdir(speaker_path):
        folder_path = os.path.join(speaker_path, folder)

        # ✅ Find the correct wavs folder
        if os.path.isdir(folder_path) and folder.endswith("_wavs"):
            wav_file_path = os.path.join(folder_path, file_name)

            # ✅ If the file exists, return it
            if os.path.exists(wav_file_path):
                return wav_file_path

    print(f"⚠️ Warning: File {file_name} not found in speaker {speaker_name} folder.")
    return None  # File not found


In [ ]:
from sentence_transformers import SentenceTransformer, util
import Levenshtein 

# ✅ Load a strong word embedding model (Multilingual)
model2 = SentenceTransformer("distiluse-base-multilingual-cased-v1")
def is_semantic_difference(text1, text2, threshold=0.85):
    """
    Uses word embeddings to check if two sentences are semantically different.
    If similarity is below `threshold`, flag it.
    """
    emb1 = model2.encode(text1, convert_to_tensor=True)
    emb2 = model2.encode(text2, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(emb1, emb2).item()  # Cosine similarity
    
    return similarity < threshold  # Flag if too different
def levenshtein_similarity(text1, text2):
    """Calculates Levenshtein similarity between two strings."""
    return Levenshtein.ratio(text1, text2)  # Returns a float between 0 and 1

#def transcribe_groq(audio_path):
    """Sends an audio file to Groq's Whisper Large v3 API and gets the transcript."""
    #with open(audio_path, "rb") as audio_file:
        #response = client.audio.transcriptions.create(
            #model="whisper-large-v3",
            #file=audio_file,
            #language=LANGUAGE,
        #)
    #return response.text

In [ ]:
import os
import json
import whisperx

# ✅ WhisperX Model (Use the same model you used before)
device = "cuda" if torch.cuda.is_available() else "cpu"
cache_dir = os.path.expanduser("~/.cache/whisperx-models")
model = whisperx.load_model("large-v2", device, compute_type="float16", download_root=cache_dir)

# ✅ Define the dataset path
DATASET_PATH = "m_ailabs_temp/en"
LANGUAGE = "en"
TARGET_SPEAKER = "judy"  # ✅ Only process "judy"

# ✅ Load dataset JSON
with open("m_ailabs_dataset.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# ✅ Function to find correct WAV file for Judy only
def find_wav_file(file_name, dataset, language):
    """Finds the correct speaker folder and corresponding wav file path."""
    if file_name not in dataset[language]:
        print(f"⚠ Warning: File {file_name} not found in dataset[{language}]!")
        return None  # Skip this file

    speaker_name = dataset[language][file_name]["speaker"]
    
    # ✅ Only process Judy's folder, skip others
    if speaker_name.lower() != TARGET_SPEAKER:
        return None  # Skip non-Judy files
    
    speaker_path = os.path.join(DATASET_PATH, speaker_name)

    # ✅ Search for `_wavs` folders dynamically
    for folder in os.listdir(speaker_path):
        folder_path = os.path.join(speaker_path, folder)
        if os.path.isdir(folder_path) and folder.endswith("_wavs"):
            wav_file_path = os.path.join(folder_path, file_name)
            if os.path.exists(wav_file_path):
                return wav_file_path

    print(f"⚠️ Warning: File {file_name} not found for speaker {speaker_name}.")
    return None  # File not found

# ✅ Process only Judy's files
flagged_files = []

for i, file_name in enumerate(dataset[LANGUAGE]):
    file_path = find_wav_file(file_name, dataset, LANGUAGE)

    if file_path is None:
        continue  # Skip non-Judy files

    if not os.path.exists(file_path):
        print(f"⚠ ERROR: File {file_path} not found, skipping...")
        continue

    # ✅ Run WhisperX Transcription
    audio = whisperx.load_audio(file_path)
    result = model.transcribe(audio, language=LANGUAGE)

    whisper_transcript = result["segments"][0]["text"] if result["segments"] else ""
    original_transcript = dataset[LANGUAGE][file_name]["text"]

    # ✅ Compare & flag mismatches
    if whisper_transcript.lower() != original_transcript.lower():
        flagged_files.append({
            "file": file_name,
            "whisper_transcript": whisper_transcript,
            "original_transcript": original_transcript
        })

    print(f"🔍 Processed {i+1} files for Judy...", end="\r")

# ✅ Save flagged files
with open("flagged_transcriptions_judy.json", "w", encoding="utf-8") as f:
    json.dump(flagged_files, f, ensure_ascii=False, indent=4)

print("\n✅ WhisperX test completed for Judy.")
print(f"🚩 {len(flagged_files)} files flagged for review.")


### 3-) Remove Noise from wav Records

#### Visualize random wavs for Noise detection

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import random
import os

# ✅ Step 1: Select a random sample from the dataset
DATASET_PATH = "m_ailabs_temp/de"  # Change to "en" when testing English
speakers = os.listdir(DATASET_PATH)
selected_speaker = random.choice(speakers)  # Pick a random speaker
speaker_path = os.path.join(DATASET_PATH, selected_speaker)

# ✅ Find a valid `_wavs` folder for the selected speaker
wavs_folders = [f for f in os.listdir(speaker_path) if f.endswith("_wavs")]
selected_wav_folder = os.path.join(speaker_path, random.choice(wavs_folders))

# ✅ Pick a random .wav file
wav_files = [f for f in os.listdir(selected_wav_folder) if f.endswith(".wav")]
selected_wav = os.path.join(selected_wav_folder, random.choice(wav_files))

# ✅ Step 2: Load the audio
y, sr = librosa.load(selected_wav, sr=None)

# ✅ Step 3: Plot Waveform
plt.figure(figsize=(12, 4))
librosa.display.waveshow(y, sr=sr)
plt.title(f"Waveform of {os.path.basename(selected_wav)}")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

plt.figure(figsize=(12, 6))
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
librosa.display.specshow(D, sr=sr, x_axis="time", y_axis="log", cmap="magma")
plt.colorbar(format="%+2.0f dB")
plt.title(f"Spectrogram of {os.path.basename(selected_wav)}")
plt.show()


In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# 🔹 Path to your processed dataset (Adjust for different speakers)
AUDIO_DIR = "m_ailabs_temp/en/elliot/elliot_pink_fairy_wavs"

# 🔹 List first 5 files in the directory
audio_files = [f for f in os.listdir(AUDIO_DIR) if f.endswith(".wav")][:5]

# 🔹 Increase the figure size for better visualization
fig, axes = plt.subplots(len(audio_files), 1, figsize=(12, 4 * len(audio_files)))

for i, file in enumerate(audio_files):
    file_path = os.path.join(AUDIO_DIR, file)
    y, sr = librosa.load(file_path, sr=None)  # Load audio

    # **Normalize the waveform amplitude separately for each file**
    y = y / np.max(np.abs(y))  # Scale amplitude between -1 and 1
    y = y * 0.9  # Make waves fill more space (but prevent clipping)

    axes[i].plot(y, color="blue", linewidth=1.8)
    axes[i].set_title(f"🔉 {file}", fontsize=14, fontweight='bold')
    axes[i].set_xlim([0, len(y)])
    axes[i].set_ylim([-1, 1])  # **Each wave now fills the full Y space**
    axes[i].set_ylabel("Amplitude", fontsize=12)

# 🔹 Reduce gaps & improve layout
plt.tight_layout()
plt.show()


#### Remove 5 wavs noise and compare them with original

In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf

# Paths and file selection
SOURCE_DIR = "m_ailabs_temp/en/elliot/elliot_pink_fairy_wavs"
TARGET_DIR = "m_ailabs_temp/en/test_noise"
os.makedirs(TARGET_DIR, exist_ok=True)

selected_files = [
    "pink_fairy_book_32_f000111.wav",
    "pink_fairy_book_26_f000011.wav",
    "pink_fairy_book_09_f000022.wav",
    "pink_fairy_book_19_f000009.wav",
    "pink_fairy_book_19_f000007.wav"
]

def noise_reduction(y, sr, noise_reduction_factor=0.05):
    """
    Apply noise reduction using spectral gating.
    """
    # Generate the spectral gating threshold
    stft = librosa.stft(y)
    magnitude, phase = librosa.magphase(stft)
    spectral_threshold = noise_reduction_factor * np.median(np.abs(magnitude), axis=1, keepdims=True)
    
    # Reduce noise by gating the spectrogram
    denoised_stft = np.where(magnitude > spectral_threshold, stft, 0)
    y_denoised = librosa.istft(denoised_stft)
    
    return y_denoised

def process_and_save(file_name):
    # Full paths
    source_path = os.path.join(SOURCE_DIR, file_name)
    target_path = os.path.join(TARGET_DIR, file_name)
    
    # Load audio
    y, sr = librosa.load(source_path, sr=None)
    
    # Apply noise reduction
    y_denoised = noise_reduction(y, sr)
    
    # Save denoised audio
    sf.write(target_path, y_denoised, sr)
    print(f"✅ Saved denoised file: {target_path}")
    
    return y, y_denoised, sr

# Visualization function
def visualize_waveforms(original, denoised, sr, file_name):
    plt.figure(figsize=(12, 6))
    
    # Original waveform
    plt.subplot(2, 1, 1)
    librosa.display.waveshow(original, sr=sr)
    plt.title(f"Original: {file_name}")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    
    # Denoised waveform
    plt.subplot(2, 1, 2)
    librosa.display.waveshow(denoised, sr=sr, color="orange")
    plt.title(f"Denoised: {file_name}")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    
    plt.tight_layout()
    plt.show()

# Process files and visualize
for file_name in selected_files:
    try:
        original, denoised, sr = process_and_save(file_name)
        visualize_waveforms(original, denoised, sr, file_name)
    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")


### 4-) Load Turkish Audio Corpus

In [ ]:
from datasets import load_dataset
import soundfile as sf
import os

# 1. Load the dataset
dataset_name = "erenfazlioglu/turkishvoicedataset"
dataset = load_dataset(dataset_name)

# 2. Inspect the dataset structure (optional, but good to verify)
print(dataset)
print(dataset["train"].features)

# 3. Define a function to extract and save data
def extract_audio_and_transcription(example, index, output_dir="extracted_data"):
    """Extracts audio and transcription from a dataset example."""
    audio = example["audio"]
    transcription = example["transcription"]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create filenames using the index as a unique ID
    audio_filename = os.path.join(output_dir, f"audio_{index:06d}.wav")  # Format index to 6 digits with leading zeros
    transcription_filename = os.path.join(output_dir, f"transcription_{index:06d}.txt")

    # Save the audio
    sf.write(audio_filename, audio["array"], audio["sampling_rate"])

    # Save the transcription
    with open(transcription_filename, "w", encoding="utf-8") as f:
        f.write(transcription)

    return audio_filename, transcription_filename


# 4. Process the dataset (or a subset for testing)
output_directory = "extracted_turkish_audio"
# Process the first 100 examples for testing
extracted_files = []

# We need to use enumerate() correctly to pass the index
for i, example in enumerate(dataset["train"]):
   
    try:
        audio_file, transcription_file = extract_audio_and_transcription(example, i, output_directory) # Pass the index!
        extracted_files.append((audio_file, transcription_file))
        print(f"Extracted example {i+1}: Audio - {audio_file}, Transcription - {transcription_file}")
    except Exception as e:
        print(f"Error processing example {i}: {e}")
        continue

print(f"\nExtracted {len(extracted_files)} audio and transcription files to '{output_directory}'.")

In [ ]:
from datasets import load_dataset
import soundfile as sf
import os

# 1. Load the dataset
dataset_name = "rakshya34/filtered_english_female_voice_v1"
dataset = load_dataset(dataset_name)

# 2. Inspect the dataset structure (optional, but good to verify)
print(dataset)
print(dataset["train"].features)

# 3. Define a function to extract and save data
def extract_audio_and_transcription(example, index, output_dir="extracted_data"):
    """Extracts audio and transcription from a dataset example."""
    audio = example["audio"]
    # Changed this line: Use "text" instead of "transcription"
    transcription = example["text"]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create filenames using the index as a unique ID
    audio_filename = os.path.join(output_dir, f"audio_{index:06d}.wav")  # Format index to 6 digits with leading zeros
    transcription_filename = os.path.join(output_dir, f"transcription_{index:06d}.txt")

    # Save the audio
    sf.write(audio_filename, audio["array"], audio["sampling_rate"])

    # Save the transcription
    with open(transcription_filename, "w", encoding="utf-8") as f:
        f.write(transcription)

    return audio_filename, transcription_filename


# 4. Process the dataset (or a subset for testing)
output_directory = "extracted_turkish_audio"
# Process the first 100 examples for testing
extracted_files = []

# We need to use enumerate() correctly to pass the index
for i, example in enumerate(dataset["train"]):
   
    try:
        audio_file, transcription_file = extract_audio_and_transcription(example, i, output_directory) # Pass the index!
        extracted_files.append((audio_file, transcription_file))
        print(f"Extracted example {i+1}: Audio - {audio_file}, Transcription - {transcription_file}")
    except Exception as e:
        print(f"Error processing example {i}: {e}")
        continue

print(f"\nExtracted {len(extracted_files)} audio and transcription files to '{output_directory}'.")

### 5-) Extract Turkish audio from mozilla Corpora 

In [ ]:
import os
import pandas as pd

# ✅ Define paths
DATASET_DIR = "tr"  # Make sure this is the correct path
AUDIO_DIR = os.path.join(DATASET_DIR, "clips")
TSV_FILES = ["train.tsv", "test.tsv", "other.tsv"]  # These should be in `tr/`

# ✅ Check if TSV files exist
for tsv_file in TSV_FILES:
    tsv_path = os.path.join(DATASET_DIR, tsv_file)
    if not os.path.exists(tsv_path):
        print(f"❌ ERROR: {tsv_file} not found in {DATASET_DIR}!")
        exit(1)

# ✅ Function to filter valid audio samples
def filter_valid_samples(tsv_path):
    """Filters samples with high up_votes and low down_votes."""
    df = pd.read_csv(tsv_path, sep="\t")

    # ✅ Keep only samples with at least 2 upvotes and 0 downvotes
    filtered_df = df[(df["up_votes"] >= 2) & (df["down_votes"] == 0)]

    print(f"✅ {len(filtered_df)} valid samples found in {os.path.basename(tsv_path)}")
    return filtered_df

# ✅ Process all TSV files
for tsv_file in TSV_FILES:
    tsv_path = os.path.join(DATASET_DIR, tsv_file)
    filtered_df = filter_valid_samples(tsv_path)

    # ✅ Save the cleaned dataset
    output_path = os.path.join(DATASET_DIR, f"filtered_{tsv_file}")
    filtered_df.to_csv(output_path, sep="\t", index=False)
    print(f"✅ Filtered data saved to: {output_path}")


In [ ]:
import os
import pandas as pd
import librosa
import soundfile as sf
from tqdm import tqdm

# ✅ Define paths
DATASET_PATH = "tr"
CLIPS_PATH = os.path.join(DATASET_PATH, "clips")  # MP3 files location
OUTPUT_PATH = os.path.join(DATASET_PATH, "processed_speakers")

# ✅ Ensure output directory exists
os.makedirs(OUTPUT_PATH, exist_ok=True)

# ✅ Load the filtered TSV files
train_tsv = pd.read_csv(os.path.join(DATASET_PATH, "filtered_train.tsv"), sep="\t")
test_tsv = pd.read_csv(os.path.join(DATASET_PATH, "filtered_test.tsv"), sep="\t")

# ✅ Combine train & test sets
filtered_data = pd.concat([train_tsv, test_tsv])

# ✅ Dictionary to track speaker folders
speaker_folders = {}

# ✅ Process each row
for _, row in tqdm(filtered_data.iterrows(), total=len(filtered_data), desc="Processing Speakers"):
    client_id, mp3_filename, _, transcription = row[:4]

    # ✅ Define speaker folder
    speaker_folder = os.path.join(OUTPUT_PATH, f"speaker_{client_id}")
    wavs_folder = os.path.join(speaker_folder, f"speaker_{client_id}_wavs")

    # ✅ Ensure directories exist
    os.makedirs(wavs_folder, exist_ok=True)

    # ✅ Convert MP3 to WAV (Store in speaker_wavs folder)
    mp3_path = os.path.join(CLIPS_PATH, mp3_filename)
    wav_filename = os.path.splitext(mp3_filename)[0] + ".wav"
    wav_path = os.path.join(wavs_folder, wav_filename)

    try:
        audio, sr = librosa.load(mp3_path, sr=24000)  # CommonVoice uses 24kHz
        sf.write(wav_path, audio, sr)

        # ✅ Store transcription
        metadata_path = os.path.join(speaker_folder, f"speaker_{client_id}_metadata.txt")
        with open(metadata_path, "a", encoding="utf-8") as f:
            f.write(f"{wav_filename}|{transcription}\n")

        # ✅ Track processed speakers
        speaker_folders[speaker_folder] = True

    except Exception as e:
        print(f"⚠ Error processing {mp3_filename}: {e}")

# ✅ Show results
num_speakers = len(speaker_folders)
print(f"✅ Processing complete! {num_speakers} unique speakers organized.")


#### remove The speakers who have less than 50 audio files

In [ ]:
import os
from collections import Counter

# ✅ Path to processed Mozilla dataset (where speakers are stored)
DATASET_PATH = "tr/processed_speakers"  # Update if needed

# ✅ Count files per speaker
speaker_counts = {spk: len(os.listdir(os.path.join(DATASET_PATH, spk, f"{spk}_wavs")))
                  for spk in os.listdir(DATASET_PATH)
                  if os.path.isdir(os.path.join(DATASET_PATH, spk))}

# ✅ Sort by most recordings
sorted_speakers = sorted(speaker_counts.items(), key=lambda x: x[1], reverse=True)

# ✅ Filtering: Only keep speakers with at least X samples
MIN_AUDIO_FILES = 50  # ✅ Set minimum threshold (change as needed)
valid_speakers = {spk: count for spk, count in sorted_speakers if count >= MIN_AUDIO_FILES}

# ✅ Print summary
print("\n🎙️ **Speaker Data Distribution (Mozilla TR)** 🎙️")
print(f"📊 Total Speakers: {len(speaker_counts)}")
print(f"✅ Speakers with >= {MIN_AUDIO_FILES} samples: {len(valid_speakers)}")
print("🔹 Top 10 Speakers by Data:")

for i, (spk, count) in enumerate(list(valid_speakers.items())[:10]):
    print(f"  {i+1}. {spk}: {count} samples")

# ✅ Decide next steps:
remaining_speakers = len(speaker_counts) - len(valid_speakers)
print(f"\n⚠️ {remaining_speakers} speakers have too few samples and will be excluded.")

# ✅ Evaluate HF dataset addition
HF_TARGET_COUNT = 20000  # ✅ Decide how much we need from HF
current_total = sum(valid_speakers.values())

print(f"\n🔍 Current total audio samples: {current_total}")
if current_total < HF_TARGET_COUNT:
    needed = HF_TARGET_COUNT - current_total
    print(f"⚡ Need to add {needed} samples from HF dataset.")

print("\n🚀 Ready for next steps!")


In [ ]:
import os
import shutil
import json

# Define dataset path
DATASET_PATH = "tr/processed_speakers/"

# Define the top 10 speakers to keep
top_speakers = {
    "speaker_ba25f19f22267b1f863cef586f17252a9067bc0ecb3d63dd7626fbce7014a16412dd64d5404d34505c0c90635adbc3cb2d830d77291a4db194e5abad91160957",
    "speaker_ca179eb54e4e584d30587ba3c6c1d4de7d9082d649113d481d308f3e7d858e0a1df21d9591f9c613928b1e122aa7f54535d0f0e0c1380d87d8017844e6e73ea2",
    "speaker_6cf4dea5df1dfe5dc5d7bacdb6da105b9e48e01fd0f069dbaf0ad906a7c5d78032194a3a9384e8589b93c58a04a12f676669f4c4501e52cee652f331b72b9205",
    "speaker_b0ec03dc36c7ef4ef29409671a91c488e78765d3a17d32b778c7e4425c32026f9341a6e73f11cda3c0cd7ed97f4e5dc3ec6b8b099098c9ddfb21345a0c39eb07",
    "speaker_60cee2235d7ec4cdeb89d601b8c373955b303c712ec729ed0affdabda8819908f51cefa990163d2bc4ac04c93e6dac2909cca67829211df4a2a17af2507dd50a",
    "speaker_c3c204ffaebfc46c0265773376f9288a372694aa79f97afe224828033c28d6d2a90919aaf769bf14105cb4033650e10275760afe250490782eae15c1d1518799",
    "speaker_509e62ded4e7e780266e54e2bb68c57f9311ded1ab57def015ff9a60de6d2efbc23adcd71e845d4c501938614bd5290d248a6202c3e7c5f7543af861b57fdcf9",
    "speaker_5d0347c54aae3292ec3a39a8bef0c8cc5e02f4c86ddfdf9dacf3a51cd8b68ca9e0f645b88ac280a1e4b25200de0771ddac89962debaf9d666b6330ac58075e40",
    "speaker_618f55a108c81342ad0f4e011b4346cc4c330158c46ef0105cc2d37a986b9025426aae64a2dc944896c5847c761f0d254b950559620f0605a5a813cf241d2c82",
    "speaker_137102afdef7dc128fc101a2c19623d189087e7e3cf388c08fa5298349682fc8e9b7130a4ae2f4d8fdf8eaa584afd95f9b1df35dff651ad53c479703f3d83ecf"
}

# List all speaker folders
all_speakers = os.listdir(DATASET_PATH)

# Remove unwanted speakers
removed_speakers = []
for speaker in all_speakers:
    speaker_path = os.path.join(DATASET_PATH, speaker)
    if os.path.isdir(speaker_path) and speaker not in top_speakers:
        shutil.rmtree(speaker_path)  # Remove entire folder
        removed_speakers.append(speaker)

# Output summary
len_removed = len(removed_speakers)
len_remaining = len(top_speakers)
{
    "removed_speakers": len_removed,
    "remaining_speakers": len_remaining,
    "total_speakers_now": len(os.listdir(DATASET_PATH))
}


#### Check some audio files Herz 

In [ ]:
import librosa
import os

# ✅ Define some sample WAV files from each dataset
samples = {
    "DE-1": "m_ailabs_temp/de/rebeca/rebeca_das_letzte_marchen_wavs/das_letzte_marchen_001_f000001.wav",
    "DE-2": "m_ailabs_temp/de/karlson/karlson_altehous_wavs/altehaus_001_f000054.wav",
    "DE-3": "m_ailabs_temp/de/ramona/ramona_alter_afrikaner_wavs/alter_afrikaner_01_f000055.wav",
    "DE-4": "m_ailabs_temp/de/eva/eva_grune_haus_wavs/grune_haus_02_f000076.wav",
    "EN-1": "m_ailabs_temp/en/mary/mary_midnight_wavs/midnight_passenger_01_f000001.wav",
    "EN-2": "m_ailabs_temp/en/elliot/elliot_hunter_space_wavs/hunters_space_03_f000041.wav",
    "EN-3": "m_ailabs_temp/en/judy/judy_dorothy_wizard_wavs/dorothy_and_wizard_oz_01_f000052.wav",
    "EN-4": "m_ailabs_temp/en/elizabeth/elizabeth_jane_eyre_wavs/jane_eyre_01_f000068.wav",
    "TR": "m_ailabs_temp/tr/ahmet/ahmet_wavs/common_voice_tr_30332345.wav",
    "TR-2": "m_ailabs_temp/tr/emre/emre_wavs/audio_045851.wav",
    "TR-3": "m_ailabs_temp/tr/emre/emre_wavs/audio_050000.wav",
    "TR-4": "m_ailabs_temp/tr/emre/emre_wavs/audio_054310.wav",
    "TR-5": "m_ailabs_temp/tr/emre/emre_wavs/audio_032427.wav",
    "TR-6": "m_ailabs_temp/tr/emre/emre_wavs/audio_033125.wav",
}

# ✅ Check the sample rates
for lang, file_path in samples.items():
    audio, sr = librosa.load(file_path, sr=None)  # ⚠️ Load with original SR
    print(f"🎙️ {lang} Sampling Rate: {sr} Hz")


In [1]:
from TTS.tts.models.xtts import Xtts
from TTS.tts.configs.xtts_config import XttsConfig

# ✅ Load XTTS config (No need to load full model)
config = XttsConfig()
print(f"🔍 XTTS Expected Sampling Rate: {config.audio.sample_rate} Hz")


/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 XTTS Expected Sampling Rate: 22050 Hz


In [ ]:
import os
import librosa
import soundfile as sf

TARGET_SR = 22050  # XTTS expected sampling rate
DATASET_PATH = "m_ailabs_temp"

def resample_audio(dataset_path, lang):
    """ Convert all audio files in dataset to 22050Hz. """
    print(f"🚀 Processing {lang} dataset...")

    # Go through all speaker folders
    for speaker in os.listdir(os.path.join(dataset_path, lang)):
        if speaker=="emre":
            speaker_path = os.path.join(dataset_path, lang, speaker)
            
            # Find the wavs folder
            for folder in os.listdir(speaker_path):
                folder_path = os.path.join(speaker_path, folder)
                
                if os.path.isdir(folder_path) and folder.endswith("_wavs"):
                    for file in os.listdir(folder_path):
                        if file.endswith(".wav"):
                            file_path = os.path.join(folder_path, file)
                            
                            # Load and check sampling rate
                            audio, sr = librosa.load(file_path, sr=None)  # Keep original SR
                            if sr != TARGET_SR:
                                print(f"🔄 Resampling {file} ({sr} Hz → {TARGET_SR} Hz)")
                                audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
                                sf.write(file_path, audio, TARGET_SR)

    print(f"✅ {lang} dataset converted to {TARGET_SR}Hz!\n")

# 🔥 Run for all languages
for lang in ["tr"]:
    resample_audio(DATASET_PATH, lang)

print("🎯 All datasets successfully standardized to 22050Hz!")


In [ ]:
# Re-run the function due to execution state reset

import os
import shutil

# ✅ Define source & destination paths
SRC_AUDIO_DIR = "extracted_turkish_audio/"
DST_SPEAKER_DIR = "m_ailabs_temp/tr/emre/"
DST_WAVS_DIR = os.path.join(DST_SPEAKER_DIR, "emre_wavs")
DST_METADATA_FILE = os.path.join(DST_SPEAKER_DIR, "emre_metadata.txt")

# ✅ Ensure destination folders exist
os.makedirs(DST_WAVS_DIR, exist_ok=True)

# ✅ Filtering criteria
MIN_WORD_COUNT = 15  # At least 15 words

# ✅ Process files
selected_samples = []
for file in os.listdir(SRC_AUDIO_DIR):
    if file.endswith(".txt"):  # Process transcription files only
        txt_path = os.path.join(SRC_AUDIO_DIR, file)
        wav_file = file.replace("transcription", "audio").replace(".txt", ".wav")
        wav_path = os.path.join(SRC_AUDIO_DIR, wav_file)

        if not os.path.exists(wav_path):  # Ensure corresponding WAV file exists
            continue

        with open(txt_path, "r", encoding="utf-8") as f:
            transcription = f.read().strip()

        # ✅ Check word count
        if len(transcription.split()) >= MIN_WORD_COUNT:
            # ✅ Move WAV file to destination
            shutil.move(wav_path, os.path.join(DST_WAVS_DIR, wav_file))
            selected_samples.append(f"{wav_file}|{transcription}")

# ✅ Write metadata file
with open(DST_METADATA_FILE, "w", encoding="utf-8") as f:
    f.write("\n".join(selected_samples))

# ✅ Output summary
num_files_moved = len(selected_samples)
DST_WAVS_DIR, DST_METADATA_FILE, num_files_moved
